# 03.1 - Transformer Models - Query

How to load and query a Hugging Face Transformer model.

In [ ]:
%pip install --quiet --upgrade transformers

The [Accelerate](https://huggingface.co/docs/accelerate/) library allows models layers to be distruibuted across hardware (GPU, CPU, disk drive) with PyTorch. It's required for using 'device_map' when loading a model.

In [ ]:
%pip install --quiet --upgrade accelerate

In [ ]:
model_id = "HuggingFaceTB/SmolLM2-1.7B-Instruct"

model_kwargs = {"torch_dtype": "auto", "device_map": "auto"}

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, **model_kwargs)

In [ ]:
import utils

utils.print_model_info(model)

In [ ]:
generate_kwargs = {
    "max_new_tokens": 1024,
    "do_sample": True,
    "temperature": 0.2,
    "top_k": 50,
    "top_p": 0.9,
    "bos_token_id": tokenizer.bos_token_id,
    "pad_token_id": tokenizer.eos_token_id,
    "eos_token_id": tokenizer.eos_token_id,
}

In [ ]:
%%time

prompt = """
What is the capital of France?
"""

messages = [{"role": "user", "content": prompt}]

input_text = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt"
).to(model.device)

outputs = model.generate(input_text, **generate_kwargs)

response = outputs[0][input_text.shape[-1] :]

print(tokenizer.decode(response, skip_special_tokens=True) + "\n")

In [ ]:
%%time

system_prompt = """
Provide a concise summary of the provided text.
"""

prompt = """
There once was a man from Nantucket,
who wore on his head a large bucket.
He was thinking one day,
that there's no rain today,
so he took his bucket then chucked it.
"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
]

input_text = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt"
).to(model.device)

outputs = model.generate(input_text, **generate_kwargs)

response = outputs[0][input_text.shape[-1] :]

print(tokenizer.decode(response, skip_special_tokens=True) + "\n")

## Using Pipelines

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model_id,
    **model_kwargs,
)

In [ ]:
prompt = """
What is the capital of France?
"""

messages = [{"role": "user", "content": prompt}]

input_text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

response = pipe(messages, max_new_tokens=512)
messages = response[0]["generated_text"]

print(messages[-1]["content"])

### Streaming output

In [ ]:
from transformers import TextStreamer

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, **model_kwargs)

streamer = TextStreamer(tokenizer, skip_special_tokens=True, skip_prompt=True)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, streamer=streamer)

In [ ]:
prompt = """
Recite the Gettysburg Address
"""

messages = [{"role": "user", "content": prompt}]

input_text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

response = pipe(messages, max_new_tokens=2048)